In [2]:
import matlab.engine as engine

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

mdl = 'transmission_fault_detection'

In [3]:
eng = engine.connect_matlab()
eng.set_param(mdl,'FastRestart','on',nargout=0)
out = eng.sim(mdl)
g1=eng.evalin('base','g1')
g2=eng.evalin('base','g2')
g3=eng.evalin('base','g3')

sensor = pd.DataFrame({'s1':[],
                       's2':[],
                       's3':[],
                       's4':[],
                       's5':[],
                       's6':[],
                       's7':[],
                       's8':[],
                       's9':[],
                       'MDPstate':[],
})

for mk_state in range(6):
    # Clutch fault injection
    if mk_state == 0:
        clutch = np.array([-1,-1,-1,-1,-1,-1],dtype=np.int64)
    elif mk_state == 1:
        clutch = np.array([1,-1,-1,-1,-1,-1],dtype=np.int64)
    elif mk_state == 2:
        clutch = np.array([-1, 1, 1,-1,-1,-1],dtype=np.int64)
    elif mk_state == 3:
        clutch = np.array([-1,-1,-1, 1, 1, 0],dtype=np.int64)
    elif mk_state == 4:
        clutch = np.array([0, 0,-1,-1, 1, 1],dtype=np.int64)
    elif mk_state == 5:
        clutch = np.array([0,0,0,0,0,0],dtype=np.int64)

    eng.set_param(mdl+'/[A B C D E F]','Value',np.array2string(clutch),nargout=0)
    out = eng.sim(mdl)
    # Get actual drive ratio
    idx = [{'type':'.','subs':'yout'},{'type':'{}','subs':[2]},{'type':'.','subs':'Values'},{'type':'.','subs':'Data'}]
    tmp_out = out
    for tmp_idx in idx:
        tmp_out = eng.subsref(tmp_out,tmp_idx)
    actual_dr = np.array(tmp_out)

    # Get actual drive ratio time
    idx = [{'type':'.','subs':'yout'},{'type':'{}','subs':[2]},{'type':'.','subs':'Values'},{'type':'.','subs':'Time'}]
    tmp_out = out
    for tmp_idx in idx:
        tmp_out = eng.subsref(tmp_out,tmp_idx)
    actual_time = np.array(tmp_out)

    # Get gear command, there are 6 gears
    idx = [{'type':'.','subs':'yout'},{'type':'{}','subs':[4]},{'type':'.','subs':'Values'},{'type':'.','subs':'Data'}]
    tmp_out = out
    for tmp_idx in idx:
        tmp_out = eng.subsref(tmp_out,tmp_idx)
    gear_command = np.array(tmp_out).reshape(actual_dr.shape[0],6)

    expected_dr = np.zeros(actual_dr.shape)
    for i in range(gear_command.shape[0]):
        # Gear R
        if (gear_command[i] == [1,0,0,1,0,1]).all():
            expected_dr[i,0] = -g1/(g2*(1+g1))
        # Gear 1
        elif (gear_command[i] == [0,0,1,1,0,1]).all():
            expected_dr[i,0] = g1/(g3*(1+g1))
        # Gear 2
        elif (gear_command[i] == [0,0,1,0,1,1]).all():
            expected_dr[i,0] = g1*(1+g2)/((1+g1)*(g3+g2))
        # Gear 3
        elif (gear_command[i] == [1,0,1,0,0,1]).all():
            expected_dr[i,0] = g1/(1+g1)
        # Gear 4
        elif (gear_command[i] == [0,1,1,0,0,1]).all():
            expected_dr[i,0] = (g3*(1+g1)-1)/(g3*(1+g1))
        # Gear 5
        elif (gear_command[i] == [1,1,1,0,0,0]).all():
            expected_dr[i,0] = 1.0
        # Gear 6
        elif (gear_command[i] == [1,1,0,0,0,1]).all():
            expected_dr[i,0] = (g2*(1+g1)+1)/(g2*(1+g1))
        # Gear 7
        elif (gear_command[i] == [0,1,0,0,1,1]).all():
            expected_dr[i,0] = (1+g2)/g2
        # Gear 0
        else: 
            expected_dr[i,0] = 0
    splid_len = 100
    for j in range(splid_len):
        tmp_idx = [np.where(actual_time<=4*i+4/splid_len*j)[0][-1] for i in range(9)]
        act_dr = actual_dr[tmp_idx]
        exp_dr = expected_dr[tmp_idx]
        tmp_sensor=pd.DataFrame(act_dr.T,columns=['s1','s2','s3','s4','s5','s6','s7','s8','s9'])
        tmp_sensor['MDPstate'] = mk_state
        sensor=pd.concat([sensor,tmp_sensor],ignore_index=True)


In [4]:
sensor.head()
# eng.quit()

,s1,s2,s3,s4,s5,s6,s7,s8,s9,MDPstate
0,0.000000,-0.261866,3.291945e-18,0.232221,0.412308,0.615385,0.854862,1.000000,1.163666,0.0
1,-0.261866,-0.261866,2.250886e-01,0.247950,0.412308,0.854862,0.854862,1.000000,1.163666,0.0
2,-0.261866,-0.261866,2.322206e-01,0.412308,0.517007,0.854862,0.870187,1.008256,1.163666,0.0
3,-0.261866,-0.261866,2.322206e-01,0.412308,0.615385,0.854862,0.968973,1.100110,1.166024,0.0
4,-0.261866,-0.238882,2.322206e-01,0.412308,0.615385,0.854862,1.000000,1.163666,1.223792,0.0


In [19]:
train_data = sensor[sensor.columns[0:9]].values
train_label = sensor['MDPstate'].values
train_label = keras.utils.to_categorical(train_label)
scaler = MinMaxScaler()
scaler.fit(train_data)
train_data = scaler.transform(train_data)

In [32]:
x_train,x_test,y_train,y_test = train_test_split(train_data, train_label, test_size=0.33)
x_train = x_train.reshape((402,1,9))
x_test = x_test.reshape((198,1,9))

In [33]:
model = keras.Sequential(
    [
        layers.LSTM(32,return_sequences=True,activation='relu',input_shape=[1,9]),
        layers.LSTM(32),
        layers.Dropout(0.2),
        layers.Dense(6,activation='sigmoid')
    ]
)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 1, 32)             5376      
                                                                 
 lstm_7 (LSTM)               (None, 32)                8320      
                                                                 
 dropout_3 (Dropout)         (None, 32)                0         
                                                                 
 dense_3 (Dense)             (None, 6)                 198       
                                                                 
Total params: 13,894
Trainable params: 13,894
Non-trainable params: 0
_________________________________________________________________


In [34]:
history = model.fit(x_train,y_train,
                    epochs=100,
                    validation_split=0.1)

Epoch 1/100
12/12 [==============================] - 3s 43ms/step - loss: 1.7913 - accuracy: 0.2244 - val_loss: 1.7878 - val_accuracy: 0.3171
Epoch 2/100
12/12 [==============================] - 0s 4ms/step - loss: 1.7857 - accuracy: 0.4377 - val_loss: 1.7807 - val_accuracy: 0.5366
Epoch 3/100
12/12 [==============================] - 0s 5ms/step - loss: 1.7787 - accuracy: 0.5014 - val_loss: 1.7713 - val_accuracy: 0.5366
Epoch 4/100
12/12 [==============================] - 0s 5ms/step - loss: 1.7685 - accuracy: 0.4792 - val_loss: 1.7566 - val_accuracy: 0.5366
Epoch 5/100
12/12 [==============================] - 0s 6ms/step - loss: 1.7522 - accuracy: 0.5042 - val_loss: 1.7326 - val_accuracy: 0.5366
Epoch 6/100
12/12 [==============================] - 0s 5ms/step - loss: 1.7246 - accuracy: 0.4848 - val_loss: 1.6901 - val_accuracy: 0.4878
Epoch 7/100
12/12 [==============================] - 0s 5ms/step - loss: 1.6785 - accuracy: 0.4986 - val_loss: 1.6242 - val_accuracy: 0.4878
Epoch 8/100


In [42]:
label_pro = model.predict(x_test)
label_pre = label_pro.argmax(axis=-1)
label_test = y_test.argmax(axis=-1)
acc = np.sum(label_pre==label_test) / label_pre.shape[0]
print("Accuracy is:", acc)

7/7 [==============================] - 0s 2ms/step
Accuracy is: 0.9848484848484849
